In [37]:
import openai
import discord
from discord.ext import commands
import asyncio
import os
import time
import requests
import re
import json
import logging
import wandb
import random
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from wandb.integration.langchain import WandbTracer
from langchain.vectorstores import Chroma


In [29]:
path = "./testdocs"   
for filename in os.listdir(path):
# check if the file is a pdf
    if filename.endswith('.pdf'):
        text = ""
        with open(os.path.join(path, filename), 'rb') as pdf_doc:
        
            pdf_reader = PdfReader(pdf_doc)
            for page in pdf_reader.pages:
                text += page.extract_text()

                print(text)
        
  

LIMA: Less Is More for Alignment
Chunting Zhou<Pengfei Liu<Puxin XuSrini IyerJiao Sun
Yuning MaoXuezhe MaAvia EfratPing YuLili YuSusan Zhang
Gargi GhoshMike LewisLuke ZettlemoyerOmer Levy
Meta AI
Carnegie Mellon University
University of Southern California
Tel Aviv University
Abstract
Largelanguagemodelsaretrainedintwostages: (1)unsupervisedpretrainingfrom
rawtext,tolearngeneral-purposerepresentations,and(2)largescaleinstruction
tuning and reinforcement learning, to better align to end tasks and user preferences.
Wemeasure therelativeimportanceofthesetwo stagesbytrainingLIMA,a65B
parameter LLaMa language model ﬁne-tuned with the standard supervised loss on
only 1,000 carefully curated prompts and responses, without any reinforcement
learningorhumanpreferencemodeling. LIMAdemonstratesremarkablystrong
performance,learningtofollowspeciﬁcresponseformatsfromonlyahandfulof
examples in the training data, including complex queries that range from planning
tripitinerariestosp

In [30]:

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_text(text)
print(len(chunks)) 

77


In [40]:
embeddings = OpenAIEmbeddings()
#embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
vectorstore = FAISS.from_texts(texts=chunks, embedding=embeddings)
db = Chroma.from_documents(chunks, embeddings)
print(vectorstore)

AttributeError: 'str' object has no attribute 'page_content'

wandb: Network error (ConnectionError), entering retry loop.


In [33]:
llm = ChatOpenAI()
#llm = HuggingFaceHub(repo_id="tiiuae/falcon-40b", model_kwargs={"temperature":0.5, "max_length":512})

memory = ConversationBufferMemory(
    memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs=dict(k=1)),
    memory=memory
)
print( conversation_chain)

memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history') callbacks=None callback_manager=None verbose=False combine_docs_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=ChatPromptTemplate(input_variables=['question', 'context'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_

In [34]:
llm = ChatOpenAI()
#if message.content.startswith('!hf'): 
#  llm = HuggingFaceHub(repo_id="tiiuae/falcon-40b", task="summarization", model_kwargs={"temperature":0.5, "max_length":1512})
#  #llm = HuggingFacePipeline.from_model_id(model_id="tiiuae/falcon-40b-instruct", task="summarization", model_kwargs={"temperature":0, "max_length":64})
#  logging.info("SETTING MODEL TO HUGGING FACE")


wandb_config = {"project": "wandb_prompts_quickstart"}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(search_kwargs=dict(k=1)),callbacks=[WandbTracer(wandb_config)])
query = "Give and extremely detailed Summary of this document, if you know know dont make anything up, include THE title  AND A LIST OF THE AUTHORS WHEN AVAILABLE and ALL the IMPORTTANT IDEAS expressed in the document as bullet points in markdown format"
answer=qa.run(query)
logging.info(answer)
print( answer)
   

# Summary of the Document: 

## Title 
The document doesn't have a specific title. It describes a dataset used for training a machine learning model for question-answering system. 

## Authors 
The document doesn't mention any specific author(s). 

## Ideas Expressed 
- WikiHow is an online wiki-style publication featuring over 240,000 how-to articles on a variety of topics. 
- Anyone can contribute to wikiHow, though articles are heavily moderated, resulting in almost universally high-quality content. 
- The dataset for training the machine learning model for question-answering system is created by sampling 200 articles from wikiHow. 
- A category is sampled first (out of 19) and then an article within it to ensure diversity. 
- The article's title is used as the prompt and the article's body is used as the response in the dataset. 
- The document explains the preprocessing heuristics applied to prune links, images, and certain sections of the text. 
- In the dataset, the typical “Thi

In [38]:



# select which embeddings we want to use
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(chunks, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,callbacks=[WandbTracer(wandb_config)])
query = "Give and extremely detailed Summary of this document, if you know know dont make anything up, include THE title  AND A LIST OF THE AUTHORS WHEN AVAILABLE and ALL the IMPORTTANT IDEAS expressed in the document as bullet points in markdown format"
answer=qa.run(query)
logging.info(answer)
print( answer)
  

AttributeError: 'str' object has no attribute 'page_content'